In [ ]:
import os
import librosa
import numpy as np
import pandas as pd

# Set seed
np.random.seed(42)

# Class labels from train audio
class_labels = sorted(os.listdir('/kaggle/input/birdclef-2025/train_audio/'))


# List of test soundscapes (only visible during submission)
test_soundscape_path = '/kaggle/input/birdclef-2025/test_soundscapes/'
test_soundscapes = [os.path.join(test_soundscape_path, afile) for afile in sorted(os.listdir(test_soundscape_path)) if afile.endswith('.ogg')]       

In [1]:
class_labels

NameError: name 'class_labels' is not defined

BirdCLEF+ 2025 Sample Submission
This is a quick run through the submission process. Test data is hidden, so we can't access it before submission. In order to make a valid submission, here's what we'll do:

Make sure we predict for all 206 classes in the train data
Load a list of test soundscapes
Process each soundscape
load audio
split into 5-second chunks
run model inference for each chunk
save predictions
Make submission csv file
Submit
Ok, so here we go.

In order to make a submission, we need to:

disable internet for this notebook (Settings --> Turn off internet)
make sure the notebook runs without errors and a submission file gets created
submit to competition (panel on the right)
wait for the notebook to finish (this may take a while, remember there's a 90-min time limit)
If all goes well, we should see our submission scores on the leaderboard.

In [ ]:
# Open each soundscape and make predictions for 5-second segments
# Use pandas df with 'row_id' plus class labels as columns
predictions = pd.DataFrame(columns=['row_id'] + class_labels)
for soundscape in test_soundscapes:

    # Load audio
    sig, rate = librosa.load(path=soundscape, sr=None)

    # Split into 5-second chunks
    chunks = []
    for i in range(0, len(sig), rate*5):
        chunk = sig[i:i+rate*5]
        chunks.append(chunk)
        
    # Make predictions for each chunk
    for i, chunk in enumerate(chunks):
        
        # Get row id  (soundscape id + end time of 5s chunk)      
        row_id = os.path.basename(soundscape).split('.')[0] + f'_{i * 5 + 5}'
        
        # Make prediction (let's use random scores for now)
        # scores = model.predict...
        scores = np.random.rand(len(class_labels))
        
        # Append to predictions as new row
        new_row = pd.DataFrame([[row_id] + list(scores)], columns=['row_id'] + class_labels)
        predictions = pd.concat([predictions, new_row], axis=0, ignore_index=True)

In [ ]:
pip install kaggle

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.model_download("google/bird-vocalization-classifier/tensorFlow2/bird-vocalization-classifier")

print("Path to model files:", path)

In [ ]:
# Save prediction as csv
predictions.to_csv('submission.csv', index=False)
predictions.head()